# Friday (prototype 2)
A dynamic NN based assistant and framework

rewritten with a Dynamic NN framework to make my life a little easier. If you would like to see a half built version in tensorflow checkout friday.ipynb

In [2]:
#imports
import torch

Handy variables

In [3]:
input_shape = [3, 3] # the input shape used in this notebook
output_shape = [3, 3] # the output shape used in this notebook
topic_shape = [3, 3]
zeros = torch.zeros(input_shape)
ones = torch.ones(input_shape)